In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

tf.__version__

'2.10.0'

ZOO classification
Data list
동물 이름 animal name: (deleted)
털 hair Boolean
깃털 feathers Boolean
알 eggs Boolean
우유 milk Boolean
날 수있는지 airborne Boolean
수중 생물 aquatic Boolean
포식자 predator Boolean
이빨이 있는지 toothed Boolean
척추 동물 backbone Boolean
호흡 방법 breathes Boolean
독 venomous Boolean
물갈퀴 fins Boolean
다리 legs Numeric (set of values: {0",2,4,5,6,8})
꼬리 tail Boolean
사육 가능한 지 domestic Boolean
고양이 크기인지 catsize Boolean
동물 타입 type Numeric (integer values in range [0",6])

In [3]:
xy = np.loadtxt('data-04-zoo.csv',
                delimiter=',',
                dtype=np.int32)
x_train = xy[0:-10, 0:-1]
y_train = xy[0:-10, [-1]]

x_train = tf.cast(x_train, tf.float32)

x_test = xy[-10:, 0:-1]
y_test = xy[-10:, [-1]]

x_test = tf.cast(x_test, tf.float32)

nb_classes = 7

y_train = tf.one_hot(list(y_train), nb_classes)
y_train = tf.reshape(y_train, [-1, nb_classes])
print(y_train[15])

y_test = tf.one_hot(list(y_test), nb_classes)  # one hot 으로 변환
print(f"before reshape:{y_test.shape}")
y_test = tf.reshape(y_test, [-1, nb_classes])

tf.Tensor([0. 0. 0. 0. 0. 0. 1.], shape=(7,), dtype=float32)
before reshape:(10, 1, 7)


In [4]:
dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(len(x_train))

W = tf.Variable(tf.random.normal([16, nb_classes], 0, 1), name='weight')
b = tf.Variable(tf.random.normal([nb_classes], 0, 1), name='bias')

print(W.shape, b.shape)
print(len(x_train))

(16, 7) (7,)
91


In [19]:
def logistic_regression(features):  # hypothesis_softmax
    return tf.nn.softmax(tf.matmul(features, W) + b)


print(tf.reduce_mean(logistic_regression(x_train)))

tf.Tensor(0.14285715, shape=(), dtype=float32)


In [6]:
def loss_fn(hypothesis, labels):
    #loss = -tf.reduce_mean(tf.reduce_sum(labels * tf.math.log(hypothesis) +(1 - labels) * tf.math.log(1 - hypothesis), axis=1))
    loss = tf.keras.losses.categorical_crossentropy(labels, hypothesis)
    return loss


optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

In [15]:
epochs = 501

for step in range(epochs):
    for features, labels in dataset:
        with tf.GradientTape() as tape:
            pred = logistic_regression(features)
            loss_value = loss_fn(pred, labels)
        grads = tape.gradient(loss_value, [W, b])
        optimizer.apply_gradients(grads_and_vars=zip(grads, [W, b]))
        if step % 100 == 0:
            print("Iter: {}, Loss: {}".format(step, tf.reduce_mean(
                loss_fn(logistic_regression(features), labels))))


Iter: 0, Loss: 0.00803410541266203
Iter: 100, Loss: 0.007538143079727888
Iter: 200, Loss: 0.007096779067069292
Iter: 300, Loss: 0.00670600077137351
Iter: 400, Loss: 0.006355990190058947
Iter: 500, Loss: 0.006041437853127718


In [16]:
def accuracy_fn(hypothesis, labels):
    hypothesis = tf.argmax(hypothesis, 1)
    predicted = tf.cast(hypothesis, dtype=tf.float32)
    print(predicted)

    labels = tf.argmax(labels, 1)
    labels = tf.cast(labels, dtype=tf.float32)
    print(labels)

    accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, labels), dtype=tf.float32))

    return accuracy

In [17]:
tesr_acc = accuracy_fn(logistic_regression(x_test), y_test)
print("Testset Accuracy: {:.4f}".format(tesr_acc))

tf.Tensor([2. 3. 0. 0. 1. 0. 5. 0. 6. 1.], shape=(10,), dtype=float32)
tf.Tensor([2. 3. 0. 0. 1. 0. 5. 0. 6. 1.], shape=(10,), dtype=float32)
Testset Accuracy: 1.0000


In [18]:
pred = logistic_regression(x_test)
print(tf.argmax(pred, 1))
print(tf.argmax(y_test, 1))

tf.Tensor([2 3 0 0 1 0 5 0 6 1], shape=(10,), dtype=int64)
tf.Tensor([2 3 0 0 1 0 5 0 6 1], shape=(10,), dtype=int64)
